<a href="https://colab.research.google.com/github/Jun0S2/AIBootcamp/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key="...")

블로그 글 fetch

In [23]:
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document  # Import Document
# Fetch the page content directly
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
response = requests.get(url)
response.encoding = "utf-8"  # Force UTF-8 encoding

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find the desired section
content = soup.find("div", class_="editedContent")

# Extract titles and paragraphs
docs = []  # To store parsed documents
if content:
    titles = content.find_all("h2")  # Find project titles
    paragraphs = content.find_all("p")  # Find project descriptions

    # Combine titles and paragraphs into documents
    for title in titles:
        # Try to find the closest following paragraph for each title
        paragraph = title.find_next_sibling("p")
        if paragraph:
            doc = {
                "title": title.text.strip(),
                "content": paragraph.text.strip()
            }
            docs.append(doc)

# Convert docs to Document objects
documents = [
    Document(page_content=doc['content'], metadata={"title": doc['title']})
    for doc in docs
]
# Check loaded documents
print(f"Loaded documents: {docs}")


Loaded documents: [{'title': '', 'content': '서비스 제작자: 다나와(김다애, 박나경)'}, {'title': '🏆\xa0대상', 'content': '서비스 제작자: 다나와(김다애, 박나경)'}, {'title': '[Lexi Note] 언어공부 필기 웹 서비스', 'content': '서비스 제작자: 다나와(김다애, 박나경)'}, {'title': '', 'content': '서비스 제작자: 인트(배정연, 한지수)'}, {'title': '🎖️\xa0우수상', 'content': '서비스 제작자: 인트(배정연, 한지수)'}, {'title': '[우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스', 'content': '서비스 제작자: 인트(배정연, 한지수)'}, {'title': '', 'content': '서비스 제작자: This is 스파게티!!!(박지성, 김서원, 박범수)'}, {'title': '🎖️\xa0우수상', 'content': '서비스 제작자: This is 스파게티!!!(박지성, 김서원, 박범수)'}, {'title': '[에코 클래스룸] 수업 실시간 소통 서비스', 'content': '서비스 제작자: This is 스파게티!!!(박지성, 김서원, 박범수)'}, {'title': '', 'content': '서비스 제작자: 동학대학운동(김민아, 임경진, 신은혜, 고수)'}, {'title': '🏅\xa0입선', 'content': '서비스 제작자: 동학대학운동(김민아, 임경진, 신은혜, 고수)'}, {'title': '[Crewing] 연합동아리 정보 플랫폼', 'content': '서비스 제작자: 동학대학운동(김민아, 임경진, 신은혜, 고수)'}, {'title': '', 'content': '서비스 제작자: 아이칼F4(조민제, 이민기, 강건, 박근우)'}, {'title': '🏅\xa0입선', 'content': '서비스 제작자: 아이칼F4(조민제, 이민기, 강건, 박근우)'}, {'title': '[

In [ ]:
# loader = WebBaseLoader(
#     web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer("div", class_="editedContent")  # editedContent 가 div class name인듯 함
#     ),
# )
# # Load the documents from the blog
# try:
#     docs = loader.load()
# except Exception as e:
#     print(f"Error loading documents: {e}")
#     docs = []
# # Check
# print(f"Loaded documents: {docs}")

Loaded documents: [Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='ì½”ë”©ì�€ ë�” ì�´ìƒ� ê°œë°œì��ë§Œì�˜ ì˜�ì—\xadì�´ ì•„ë‹™ë‹ˆë‹¤. ëˆ„êµ¬ë‚˜ ì•„ì�´ë””ì–´ë§Œ ì�ˆë‹¤ë©´ ì°½ì�˜ì\xa0�ì�¸ ì„œë¹„ìŠ¤ë¥¼ ë§Œë“¤ì–´ ì„¸ìƒ�ì�„ ë°”ê¿€ ìˆ˜ ì�ˆìŠµë‹ˆë‹¤. ìŠ¤íŒŒë¥´íƒ€ì½”ë”©í�´ëŸ½ì—�ì„œëŠ” ì�´ëŸ¬í•œ ê°€ëŠ¥ì„±ì�„ ë¯¿ê³\xa0, ëˆ„êµ¬ë‚˜ ì½”ë”©ì�„ í†µí•´ ì��ì‹\xa0ì�˜ ì•„ì�´ë””ì–´ë¥¼ ì‹¤í˜„í•˜ê³\xa0 ì‹¤ì\xa0œ ë¬¸ì\xa0œë¥¼ í•´ê²°í•˜ëŠ” ê²½í—˜ì�„ ìŒ“ì�„ ìˆ˜ ì�ˆë�„ë¡� ë‹¤ì–‘í•œ í”„ë¡œê·¸ë�¨ì�„ ë§ˆë\xa0¨í•˜ê³\xa0 ì�ˆìŠµë‹ˆë‹¤.<All-in> ì½”ë”© ê³µëª¨ì\xa0„ì�€ ëŒ€í•™ìƒ�ë“¤ì�´ ìº\xa0í�¼ìŠ¤ì—�ì„œ ê²ªì�€ ë¶ˆí�¸í•¨ê³¼ ë¬¸ì\xa0œë¥¼ ì��ì‹\xa0ë§Œì�˜ ì•„ì�´ë””ì–´ë¡œ í•´ê²°í•´ë³´ëŠ” ëŒ€íšŒì˜€ëŠ”ë�°ìš”. ì�´ë²ˆ ê³µëª¨ì\xa0„ì—�ì„œ ë‹¤ì–‘í•œ í˜�ì‹\xa0ì\xa0�ì�¸ ì•„ì�´ë””ì–´ì™€ ì—´ì\xa0•ìœ¼ë¡œ ê°€ë“�í•œ ìˆ˜ë§�ì�€ í”„ë¡œì\xa0�íŠ¸ê°€ íƒ„ìƒ�í–ˆìŠµë‹ˆë‹¤. ê·¸ì¤‘ ë›°ì–´ë‚œ ì„±ê³¼ë¥¼ ë‚¸ ìˆ˜ìƒ�ì�‘ 6ê°œë¥¼ ì†Œê°œí•©ë‹ˆë‹¤.ğŸ�†Â\xa0ëŒ€ìƒ�[Lexi Note] ì–¸ì–´ê³µë¶€ í•„ê¸° ì›¹ ì„œë¹„ìŠ

```
bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("div", class_="editedContent")  # editedContent 가 div class name인듯 함
    )
```

로 run 했을 때, 인코딩이 깨져서 bs4 사용함.

Split

In [24]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

# Store the split chunks in a vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key="...")
)


In [25]:
# Define a query for the vectorstore
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retriever = vectorstore.as_retriever()
retrieved_docs = retriever.invoke(user_msg)

# Function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Print formatted documents
formatted_docs = format_docs(retrieved_docs)
print("Formatted Retrieved Documents:")
print(formatted_docs)

Formatted Retrieved Documents:
<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.

<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.

코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.

서비스 제작자: 인트(배정연, 한지수)


### [Feedback]
<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.
🏆\xa0대상 : 다나와
...

이런식이 아니라 왜 위와 같은 형태로 요약이 되는 것일까요..?